O que fazer:
- em IMDbBasics, adicionar os títulos (tconst) dos episódios (IMDbEpisode) (PS: averiguar runtimeMinutes)
- em NetflixIMDb, mover os atributos de episódio em IMDbEpisode

Power BI:
- Adicionar atributo de weekday
- Adicionar atributo de ano (para saber os 3 títulos em cada ano)

# Importa as bibliotecas da aplicação

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 43.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b3f79af5eb58ac1368fa01976105b8fec814c609d409607460b67fb31ab35b4d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [24]:
import sys

from IPython.display import display
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from IPython.display import HTML, display
import tabulate

import pandas as pd
import numpy as np
import re

from google.colab import files

pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', None)

# Configura o ambiente de execução do Apache Spark

In [3]:
spark = SparkSession \
        .builder \
        .master("local[2]") \
        .appName("Netflix Analysis") \
        .config("spark.executor.cores", "2") \
        .config("spark.executor.memory", "2g") \
        .config("spark.cores.max", "4") \
        .getOrCreate()

# Realiza a leituras dos dados

In [4]:
!wget https://datasets.imdbws.com/title.akas.tsv.gz

--2022-03-09 18:15:24--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.138.111, 13.225.138.99, 13.225.138.63, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.138.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 265385945 (253M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 253.09M   141MB/s    in 1.8s    

2022-03-09 18:15:26 (141 MB/s) - ‘title.akas.tsv.gz’ saved [265385945/265385945]



In [5]:
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-03-09 18:15:26--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.138.111, 13.225.138.99, 13.225.138.63, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.138.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6096912 (5.8M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz’

title.ratings.tsv.g 100%[===================>]   5.81M  --.-KB/s    in 0.09s   

2022-03-09 18:15:26 (67.9 MB/s) - ‘title.ratings.tsv.gz’ saved [6096912/6096912]



In [6]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz

--2022-03-09 18:15:26--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.138.111, 13.225.138.99, 13.225.138.63, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.138.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153234923 (146M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 146.14M  77.3MB/s    in 1.9s    

2022-03-09 18:15:28 (77.3 MB/s) - ‘title.basics.tsv.gz’ saved [153234923/153234923]



In [7]:
!wget https://datasets.imdbws.com/title.episode.tsv.gz

--2022-03-09 18:15:28--  https://datasets.imdbws.com/title.episode.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.138.111, 13.225.138.99, 13.225.138.63, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.138.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35550129 (34M) [binary/octet-stream]
Saving to: ‘title.episode.tsv.gz’

title.episode.tsv.g 100%[===================>]  33.90M  29.5MB/s    in 1.1s    

2022-03-09 18:15:30 (29.5 MB/s) - ‘title.episode.tsv.gz’ saved [35550129/35550129]



In [8]:
!gzip -d title.akas.tsv.gz

In [9]:
!gzip -d title.ratings.tsv.gz

In [10]:
!gzip -d title.basics.tsv.gz

In [11]:
!gzip -d title.episode.tsv.gz

In [12]:
!ls

sample_data	title.basics.tsv   title.ratings.tsv
title.akas.tsv	title.episode.tsv


In [13]:
# IMDb Title Akas DataFrame
akas_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.akas.tsv")

In [14]:
# IMDb Title Ratings DataFrame
ratings_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.ratings.tsv")

In [15]:
# IMDb Title Basics DataFrame
basics_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.basics.tsv")

In [16]:
# IMDb Title Episode DataFrame
episode_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.episode.tsv")

In [18]:
# Netflix Streaming History DataFrame
netflix_df = pd.read_csv('https://raw.githubusercontent.com/vichShir/netflix-analysis/master/data/Netflix/NetflixViewingHistory2022.csv')

# Confere a estrutura dos dados

In [ ]:
print(type(akas_df))
print(type(ratings_df))
print(type(basics_df))
print(type(episode_df))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
akas_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)



In [ ]:
print(f'DataFrame Title Akas tem {akas_df.count()} linhas.')

DataFrame Title Akas tem 31268700 linhas.


In [ ]:
ratings_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [ ]:
print(f'DataFrame Title Ratings tem {ratings_df.count()} linhas.')

DataFrame Title Ratings tem 1221670 linhas.


In [ ]:
basics_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [ ]:
print(f'DataFrame Title Basics tem {basics_df.count()} linhas.')

DataFrame Title Basics tem 8750730 linhas.


In [ ]:
episode_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: string (nullable = true)
 |-- episodeNumber: string (nullable = true)



In [ ]:
print(f'DataFrame Title Episode tem {episode_df.count()} linhas.')

DataFrame Title Episode tem 6552731 linhas.


In [ ]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   450 non-null    object
 1   Date    450 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


In [ ]:
sum(netflix_df.duplicated())

0

# Pré-visualização dos dados

In [ ]:
# IMDb Title Akas DataFrame
display(akas_df.limit(5).toPandas())

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [ ]:
# IMDb Title Ratings DataFrame
display(ratings_df.limit(5).toPandas())

,tconst,averageRating,numVotes
0,tt0000001,5.7,1864
1,tt0000002,6.0,244
2,tt0000003,6.5,1632
3,tt0000004,5.8,158
4,tt0000005,6.2,2458


In [ ]:
# IMDb Title Basics DataFrame
display(basics_df.limit(5).toPandas())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [ ]:
# IMDb Title Episode DataFrame
display(episode_df.limit(5).toPandas())

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0020666,tt15180956,1,2
1,tt0020829,tt15180956,1,1
2,tt0021166,tt15180956,1,3
3,tt0021612,tt15180956,2,2
4,tt0021655,tt15180956,2,5


In [ ]:
# Netflix Streaming History DataFrame
netflix_df.head()

,Title,Date
0,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação da Itália,14/02/2022
1,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha de Kursk,08/02/2022
2,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação do Norte da África,31/01/2022
3,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha do Atlântico,30/12/2021
4,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Dunquerque,27/12/2021


# Separar o título e o nome do episódio

In [ ]:
netflix_df['Title']

In [ ]:
# Remove duplicated row streaming error
netflix_df = netflix_df[netflix_df['Title'] != 'Mars: Season 1_hook_04_16x9']

In [ ]:
def split_title(title):
  """
    Return the main title and episode name by title
  """
  main_title = title
  episode_name = ''
  texts = re.split('Temporada |Minissérie', title)

  # title have the separator key
  if len(texts) > 1:
    # get the main title
    main_title = texts[0]
    # remove white spaces
    main_title = main_title.strip()
    # remove the ':' character
    main_title = main_title[:-1]

    # get the episode name
    episode_name = texts[1]

  return (main_title, episode_name)

In [ ]:
# Get episode name from Title
netflix_df['Episode'] = netflix_df['Title'].map(lambda x: split_title(x)[1])

In [ ]:
# Update Title to the main title
netflix_df['Title'] = netflix_df['Title'].map(lambda x: split_title(x)[0])

In [ ]:
netflix_df.head()

In [ ]:
netflix_df.sample(5)

# Associar um ID do IMDb

In [ ]:
# Get unique titles
titles = netflix_df['Title'].unique()
titles = np.sort(titles)
titles

In [ ]:
print(f'São ao todo {len(titles)} títulos.')

In [ ]:
# Registering a table
akas_df.registerTempTable("akas_df")
ratings_df.registerTempTable("ratings_df")
basics_df.registerTempTable("basics_df")
episode_df.registerTempTable("episode_df")

In [ ]:
original_titles = np.array(['Toki o kakeru shôjo', 'Sen to Chihiro no kamikakushi', 'Koe no katachi', 'Anohana: The Flower We Saw That Day',
                            'Atlantis: The Lost Empire', 'SpongeBob SquarePants', 'The SpongeBob Movie: Sponge Out of Water', 'Challenger: The Final Flight',
                            'Coraline', 'Wolf Children', 'Inside Out', 'School of Rock',
                            'Coronavirus, Explained', 'Exterminadores do Além Contra a Loira do Banheiro', 'Greatest Events of WWII in Colour', 'History 101',
                            'Kubo and the Two Strings', 'Little Witch Academia', 'Little Witch Academia: The Enchanted Parade', 'Luzes no Céu: Fireworks',
                            'The Martian', 'Tonari no Totoro', 'Mission Control: The Unsung Heroes of Apollo', 'Nova', 'Nova',
                            'The Universe', 'A Whisker Away', 'One Punch Man: Wanpanman', 'ParaNorman',
                            'The Secret Life of Pets', 'Sangatsu no Lion', 'WWII in Color: Road to Victory', 'Sherlock',
                            'Flavors of Youth', 'Space Force', 'Karakai Jouzu no Takagi-san', 'The Search for Life in Space',
                            'The Walking Dead', 'Toradora!', 'Tron: Legacy', 'Full House',
                            'The Fresh Prince of Bel-Air', 'Children Who Chase Lost Voices', 'Kimi no na wa.'])
original_titles

In [ ]:
mapped_titles = dict(zip(titles, original_titles))
mapped_titles

In [ ]:
netflix_df['Title'].replace(mapped_titles, inplace=True)
netflix_df.sample(5)

In [ ]:
from IPython.display import clear_output

def get_id_from_imdb(title_name):
  titles = spark.sql(f"SELECT titleId FROM akas_df WHERE title = '{title_name}' AND types = 'imdbDisplay'").toPandas()
  titles = titles.drop_duplicates(subset=['titleId'])
  titles = titles.reset_index(drop=True)

  """
  ids_size = len(titles.index)

  if ids_size > 1:
    clear_output(wait=True)
    print(f'Escolha um ID (index) para <<{title_name}>>:')
    display(titles)
    input_id = int(input('>> '))
    id = titles['titleId'][titles.index[input_id]]
  elif ids_size == 1:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None
  """

  if len(titles) > 0:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None

  title_id = id if id != None else np.nan
  return title_id

In [ ]:
title_ids = [get_id_from_imdb(x) for x in original_titles]
title_ids

## Associar IDs nos dados do Netflix

In [ ]:
def categorise(row):
  for i in range(len(original_titles)):
    if(row['Title'] == original_titles[i]):
      return title_ids[i]
  return np.nan

In [ ]:
netflix_df['titleId'] = netflix_df.apply(lambda row: categorise(row), axis=1)
netflix_df.head()

## Tratar o nome dos episódios

In [ ]:
# Extract season number
netflix_df['seasonNumber'] = netflix_df['Episode'].str.extract(r'^[^\d]*(\d+)')
netflix_df.head()

In [ ]:
# Remove special characters at the beginning
netflix_df['Episode'] = netflix_df['Episode'].map(lambda x: x[2:].strip())
netflix_df.head()

## Imputar os episódios do título "Nova"

In [ ]:
netflix_df.loc[132,'Episode'] = 'Death Dive to Saturn'
netflix_df.loc[133,'Episode'] = 'Black Hole Apocalypse'

In [ ]:
netflix_df.loc[[132, 133]]

## Organizar colunas

In [ ]:
netflix_df = netflix_df[['Date', 'titleId', 'Title', 'Episode', 'seasonNumber']]
netflix_df.head()

In [ ]:
netflix_df.sample(5)

## Exportar dados

In [ ]:
netflix_df.to_csv('NetflixIMDb.csv', index=False, encoding='utf-8') 
files.download('NetflixIMDb.csv')

# Exportar dados do IMDb

In [ ]:
imdb_ratings = spark.sql(f"SELECT * FROM ratings_df WHERE tconst IN {tuple(title_ids)}").toPandas()
imdb_ratings.shape

(41, 3)

In [ ]:
imdb_basics = spark.sql(f"SELECT * FROM basics_df WHERE tconst IN {tuple(title_ids)}").toPandas()
imdb_basics.shape

(43, 9)

In [ ]:
imdb_episode = spark.sql(f"SELECT * FROM episode_df WHERE parentTconst IN {tuple(title_ids)}").toPandas()
imdb_episode.shape

(1879, 4)

## Download

In [ ]:
imdb_ratings.to_csv('IMDbRatings.csv', index=False, encoding='utf-8')
imdb_basics.to_csv('IMDbBasics.csv', index=False, encoding='utf-8')
imdb_episode.to_csv('IMDbEpisode.csv', index=False, encoding='utf-8')

files.download('IMDbRatings.csv')
files.download('IMDbBasics.csv')
files.download('IMDbEpisode.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Separar gêneros em IMDbBasics

In [19]:
# IMDbBasics DataFrame
imdb_basics_df = pd.read_csv('https://raw.githubusercontent.com/vichShir/netflix-analysis/master/data/IMDb/IMDbBasics.csv')

In [22]:
imdb_basics_df = imdb_basics_df.drop('genres', axis=1)\
                          .join(imdb_basics_df['genres']
                          .str.split(',', expand=True)
                          .stack().reset_index(level=1, drop=True)
                          .rename('genres'))
imdb_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Crime
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Drama
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Horror
1,tt0073174,movie,Inside Out,Inside Out,0,1975,\N,97,Comedy
1,tt0073174,movie,Inside Out,Inside Out,0,1975,\N,97,Crime


In [25]:
imdb_basics_df.to_csv('IMDbBasics.csv', index=False, encoding='utf-8')
files.download('IMDbBasics.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Finalizar sessão do Spark

In [ ]:
spark.stop()

# Área de Testes

In [ ]:
# Registering a table
akas_df.registerTempTable("akas_df")
spark.sql("SELECT * FROM akas_df").show(3)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|           \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|           \N|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 3 rows



In [ ]:
df = spark.sql("SELECT * FROM akas_df WHERE titleId = 'tt16542414'").toPandas()
df

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle


In [ ]:
ratings_df.registerTempTable("ratings_df")
spark.sql("SELECT * FROM ratings_df WHERE tconst = 'tt16542414'").show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+----------+-------------+--------+
|    tconst|averageRating|numVotes|
+----------+-------------+--------+
|tt16542414|          8.1|      86|
+----------+-------------+--------+



In [ ]:
basics_df.registerTempTable("basics_df")
spark.sql("SELECT * FROM basics_df WHERE tconst = 'tt16542414'").toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt16542414,tvEpisode,The Battle of the Atlantic,The Battle of the Atlantic,0,2021,\N,\N,"Documentary,History,War"


In [ ]:
episode_df.registerTempTable("episode_df")
spark.sql("SELECT * FROM episode_df WHERE parentTconst = 'tt16477402'").toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


,tconst,parentTconst,seasonNumber,episodeNumber
0,tt16542412,tt16477402,1,1
1,tt16542414,tt16477402,1,2
2,tt16542592,tt16477402,1,3
3,tt16542830,tt16477402,1,4
4,tt16542940,tt16477402,1,5
5,tt16543116,tt16477402,1,6
6,tt16543212,tt16477402,1,8
7,tt16543224,tt16477402,1,7
8,tt16543454,tt16477402,1,9
9,tt16543590,tt16477402,1,10
